# Content Based Filtering

The simplest form of Collaborative Filtering model is a Content Based Filtering model. In this model, we create a profile for every user based on the items that they have already rated and project both the user and the item in a shared space.

Once we have a user profile, we could use it to perform dot-product with different item representations and come up with a nearest neighbour search algorithm.

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
anime_data = pd.read_csv("../data/animes.csv")
profile_data = pd.read_csv("../data/profiles.csv")
reviews_data = pd.read_csv("../data/reviews.csv")

In [3]:
anime_data = anime_data.drop_duplicates(subset = ["uid"]).reset_index(drop = True)
anime_data.shape

(16216, 12)

In [7]:
anime_data.head(2)

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...


As we see from the above dataframe, an anime has many different features. It has both it's own categorical and numerical features. It is up to us how to create the user profiles but they must be created in a smart way.

The drawback with CB Filtering is that if we don't have a user profile for a given person, then it is not possible to serve them recommendations. And user profiles can only be created after the user has interacted with some if not all items.

Let's attempt this problem first by using the genre of an anime first and later introduce numerical features like episodes, members, score etc. to build a user profile.

# Anime-genre mapping matrix